In [11]:
# establish connection
from os import getenv
from dotenv import load_dotenv
import pyodbc
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                      "Server=localhost\\DOKUSSERVER;"
                      "Database=Lernperiode12;"
                      "Trusted_Connection=yes;")

cursor = cnxn.cursor()

In [12]:
cursor.execute("SELECT MAX(timestamp) FROM WeatherMeasurement")
last_saved_time = cursor.fetchone()[0]

In [13]:
from datetime import datetime, timedelta
if last_saved_time is None:
    last_saved_time = datetime.now() - timedelta(hours=48)
start = last_saved_time - timedelta(hours=48)
end = datetime.now()

start_str = start.date().isoformat()
end_str   = end.date().isoformat()

In [14]:
print(last_saved_time)

2025-12-09 23:00:00


In [15]:
print(start_str)

2025-12-07


In [16]:
print(end_str)

2025-12-19


In [22]:
# request
import requests
import json
from datetime import datetime

def get_weather_data(latitude, longitude):
    base_url = "https://api.open-meteo.com/v1/forecast"
    
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "hourly": [
            "temperature_2m",
            "relative_humidity_2m",
            "wind_speed_10m",
            "precipitation"
        ],
        "past_hours": 48,
        "forecast_days": 0,
        "forecast_hours": 0,
        "timezone": "auto"
    }




    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        return response.json()
    except Exception as e:
        print("Fetch failed:", e)
        return None


# Coordinates for Baden, Switzerland
baden_lat = 47.475361
baden_lon = 8.306372

weather = get_weather_data(baden_lat, baden_lon)
print(weather)

{'latitude': 47.48, 'longitude': 8.299999, 'generationtime_ms': 0.10728836059570312, 'utc_offset_seconds': 3600, 'timezone': 'Europe/Zurich', 'timezone_abbreviation': 'GMT+1', 'elevation': 394.0, 'hourly_units': {'time': 'iso8601', 'temperature_2m': '°C', 'relative_humidity_2m': '%', 'wind_speed_10m': 'km/h', 'precipitation': 'mm'}, 'hourly': {'time': ['2025-12-17T13:00', '2025-12-17T14:00', '2025-12-17T15:00', '2025-12-17T16:00', '2025-12-17T17:00', '2025-12-17T18:00', '2025-12-17T19:00', '2025-12-17T20:00', '2025-12-17T21:00', '2025-12-17T22:00', '2025-12-17T23:00', '2025-12-18T00:00', '2025-12-18T01:00', '2025-12-18T02:00', '2025-12-18T03:00', '2025-12-18T04:00', '2025-12-18T05:00', '2025-12-18T06:00', '2025-12-18T07:00', '2025-12-18T08:00', '2025-12-18T09:00', '2025-12-18T10:00', '2025-12-18T11:00', '2025-12-18T12:00', '2025-12-18T13:00', '2025-12-18T14:00', '2025-12-18T15:00', '2025-12-18T16:00', '2025-12-18T17:00', '2025-12-18T18:00', '2025-12-18T19:00', '2025-12-18T20:00', '2025

In [23]:
# convert weather data to dataframe
import pandas as pd
df = pd.DataFrame(weather['hourly'])

In [25]:
# getting only new rows with pandas
import pytz
df["time"] = pd.to_datetime(df["time"], utc=True)
last_saved_time = pd.to_datetime(last_saved_time, utc=True)
df_new = df[df["time"] > last_saved_time]

In [26]:
df_new.tail(5)

,time,temperature_2m,relative_humidity_2m,wind_speed_10m,precipitation
43,2025-12-19 08:00:00+00:00,0.6,99,4.0,0.0
44,2025-12-19 09:00:00+00:00,0.9,98,2.8,0.0
45,2025-12-19 10:00:00+00:00,1.2,97,2.3,0.0
46,2025-12-19 11:00:00+00:00,1.8,96,1.1,0.0
47,2025-12-19 12:00:00+00:00,3.1,92,1.8,0.0


In [27]:
# weather data template
SQL_API_STATEMENT = """
INSERT INTO WeatherMeasurement (
    timestamp,
    temperature,
    humidity,
    windspeed,
    precipitation
) VALUES
( ?, ?, ?, ?, ?)
"""

In [28]:
# insert current weather data
from datetime import datetime

for _, row in df_new.iterrows():
    cursor.execute(
        SQL_API_STATEMENT,
        (
            row["time"],
            row["temperature_2m"],
            row["relative_humidity_2m"],
            row["wind_speed_10m"],
            row["precipitation"]
        )
    )
cnxn.commit()


In [29]:
print(df_new.tail())
print(len(df_new))


                        time  temperature_2m  relative_humidity_2m  \
43 2025-12-19 08:00:00+00:00             0.6                    99   
44 2025-12-19 09:00:00+00:00             0.9                    98   
45 2025-12-19 10:00:00+00:00             1.2                    97   
46 2025-12-19 11:00:00+00:00             1.8                    96   
47 2025-12-19 12:00:00+00:00             3.1                    92   

    wind_speed_10m  precipitation  
43             4.0            0.0  
44             2.8            0.0  
45             2.3            0.0  
46             1.1            0.0  
47             1.8            0.0  
48


In [ ]:
# check last data time
# get data from 48 hours
    # define end and start
# compare pulled data with last inserted time
# leave only the data that is higher than max time from db
# insert
# if there is a gap
# then inform there's missing data because pulled from only 48h
# missing: x hours